In [2]:
# imports done
import itertools

# Global Declarations
transactionList = []
sdc = 0.0

# Function to read input file

def inputfile():
    del transactionList[:]
    inputFile = open(r'C:\Users\Ashwin PC\Desktop\MS Materials\Data Mining and text mining\Project - 1\check examples\inputdata.txt','r')
    transactionString = inputFile.read();
    transactionString = transactionString.replace('{','')
    transactionString = transactionString.replace('}','')
    transactionString = transactionString.replace(' ','')
    initialList = transactionString.split('\n')
    for transaction in initialList:
        transactionList.append(transaction.split(','))  

# Function to read parameter file to get sdc
def paramfilesdc():
    parameterFile = open(r'C:\Users\Ashwin PC\Desktop\MS Materials\Data Mining and text mining\Project - 1\check examples\parameter.txt','r')
    parameterString = parameterFile.read()
    misString = parameterString.split('\n')
    s = parameterString.split('SDC')[0].split('\n')
    s1 = parameterString.split('SDC')[1].split('\n')
    sdc = float(str(s1[0].split('=')[1]).strip())
    return sdc

# Function to read parameter file
def paramfile():
    param = []
    parameterFile = open(r'C:\Users\Ashwin PC\Desktop\MS Materials\Data Mining and text mining\Project - 1\check examples\parameter.txt','r')
    parameterString = parameterFile.read()
    misString = parameterString.split('\n')
    s = parameterString.split('SDC')[0].split('\n')
    s1 = parameterString.split('SDC')[1].split('\n')
    sdc = float(str(s1[0].split('=')[1]).strip())
    s.pop()
    for i in range(len(s)):
        s1 = s[i]
        s1 = s1.replace('MIS','')
        s1 = s1.replace('(','')
        s1 = s1.replace(')','')
        s[i] = s1
    mydict = dict((k.strip(), float(v.strip())) for k,v in 
                  (item.split('=') for item in s))
    s1 = parameterString.split('SDC')[1].split('\n')
    sdc = float(str(s1[0].split('=')[1]).strip())
    random = s1[1].split(':')[1].split('}')
    must_have = s1[2].split(':')[1].replace(' ', '').split('or')
    cannot_be_together = []
    for random1 in random:
        random2 = []
        random2 = random1.strip().replace('{','').replace(' ','').replace('\'','').replace(',',' ').strip().split(' ')
        if random2 == ['']:
            pass
        else:
            cannot_be_together.append(random2)
    param.append(mydict)
    param.append(sdc)
    param.append(cannot_be_together)
    param.append(must_have)
    return param

######## MS - Apriori Algorithm ##############
# Function to Sort the keys based on MIS values
def sort(combined):
    combined_List = [[k,v] for k, v in combined.items()]
    combined_List.sort(key=lambda pair: pair[1][0])
    return combined_List

# Function to generate Level-2 Candidate Generation 
def level2candidate(combined_List, l):
    c2 = []
    for j in range(len(combined_List)):
        if combined_List[j][0] in l and combined_List[j][1][1] >= combined_List[j][1][0]:
            for i in range(j+1, len(combined_List)):
                if combined_List[i][1][1] >= combined_List[j][1][0] and abs(combined_List[i][1][1] - combined_List[j][1][1]) <= sdc:
                    temp = []
                    temp.append(combined_List[j][0])
                    temp.append(combined_List[i][0])
                    c2.append(temp)
    return c2

# Function to get MIS of an element 
def getMIS(element, MIS_List):
        for i in range(len(MIS_List)):
            if element in MIS_List[i]:
                mis = MIS_List[i][1]
                break
        return mis

# Function to get support of an element
def getSupport(element, support_List):
    for i in range(len(support_List)):
        if element[0] in support_List[i]:
            support = support_List[i][1]
            break
    return support
    
# Function to generate candidate set for k > 2
def MS_CandidateGen(ftemp, MIS_List, support_List):
    fjoinlist = []
    fjointemp = []
    for i in range(len(ftemp)):
        f1 = ftemp[i][0:(len(ftemp[i])-1)]
        for j in range(i+1, len(ftemp)):
            f2 = ftemp[j][0:(len(ftemp[j])-1)]
            if f1 == f2 and abs(getSupport(ftemp[j][(len(ftemp[j])-1):], support_List) - getSupport(ftemp[i][(len(ftemp[i])-1):], support_List)) <= sdc:
                fjointemp = ftemp[i] + ftemp[j][(len(ftemp[j])-1):]
                fjoinlist.append(fjointemp)
                fjointemp = []
    i = 0
    while i < len(fjoinlist):
        subsetlist = list(set(itertools.combinations(fjoinlist[i], len(fjoinlist[i])-1)))
        for subset in subsetlist:
            if (fjoinlist[i][0] in subset) or (getMIS(fjoinlist[i][0], MIS_List) == getMIS(fjoinlist[i][1], MIS_List)):
                ftcounter = 0
                for ft in ftemp:
                    if list(subset) == list(ft):
                        ftcounter = ftcounter + 1
                        break
                if ftcounter == 0:
                    fjoinlist.pop(i)
                    i = i - 1
                    break
            if i == len(fjoinlist):
                break
        i = i + 1
    return(fjoinlist)

In [10]:
# Calling Input file
inputfile()

# Removing Empty Lists in transactionList
i=0
while(i<len(transactionList)):
    if transactionList[i] == ['']:
        transactionList.pop(i)
        i=i-1
    i=i+1
#print("transactionList")
#print(transactionList)

# Calling Parameter File to read MIS Values
param = paramfile()
mydict = param[0]
sdc = param[1]
cannot_be_together = param[2]
must_have = param[3]

# Creating Lists with MIS values and support values
support = {}
count = 0
for key in mydict:
    for transaction in transactionList:
        if key in transaction:
                count = count + 1
    support[key] = round(count/len(transactionList),2)
    count = 0
combined = dict([(k,[mydict[k],support[k]]) for k in mydict])
MIS_List = [ [k,v] for k, v in mydict.items() ]
support_List = [ [k,v] for k, v in support.items() ]
print("MIS_List")
print(MIS_List)
print("support_List")
print(support_List)

# MS - Apriori Algorithm Starts
# Step 1 - Sorting the list based on MIS values
combined_List = sort(combined)
print("combined_List with MIS and support -- Sorted based on MIS")
print(combined_List)

# Step 2: Init-Pass
l = []
f = []
f1 = []
f2 = []
f3 = []
j = -1
for i in range(len(combined_List)):
    if combined_List[i][1][1] >= combined_List[i][1][0]:
        f1.append(combined_List[i][0])
        j = i
    if j >= 0:
        if combined_List[i][1][1] >= combined_List[j][1][0]:
            l.append(combined_List[i][0])
print("l after init pass")
print(l)
i = 0
while i < len(f1):
    if f1[i] in must_have:
        pass
    else:
        f1.pop(i)
        i = i - 1
    i = i + 1
print("f1 after init pass")
print(f1)
for f1obj in f1:
    count = 0
    for transaction in transactionList:
        if f1obj in transaction:
            count = count + 1
    f2.append([f1obj,count])
f.append(f2)

# Creating the loop to run the MS-Apriori Algorithm
k = 2
ftemp = []
while(k == 2 or len(ftemp) > 1):
    if k == 2:
        # Calling level 2 candidate Generation function
        c = level2candidate(combined_List, l)
        print("Level 2 Candidate Generation")
        print(c)       
    else:
        # Calling candidate generation function for k > 2
        c = MS_CandidateGen(ftemp, MIS_List, support_List)
        print('Level K > 2 Candidate Generation')
        print(c)
    # Checking each c value and calculating f value
    count = 0
    mis = 0
    ftemp = []
    ftempwithcount = []
    ffinal = []
    for cg in c:
        for transaction in transactionList:
            if set(cg) < set(transaction):
                count = count + 1
        for i in range(len(MIS_List)):
            if cg[0] in MIS_List[i]:
                mis = MIS_List[i][1]
                break
        if count/len(transactionList) >= mis:
            ftemp.append(cg)
            ftempwithcount.append([cg,count])
        count = 0
    
    # Removing Cannot be together items
    i=0
    while i < len(ftemp):
        for eachset in cannot_be_together:
            if(set(eachset) <= set(ftemp[i])):
                ftemp.pop(i)
                ftempwithcount.pop(i)
                i = i - 1
                break
        i = i + 1

    # Removing sets without Must Have
    i=0
    while i < len(ftemp):
        must_have_counter = 0
        for eachvalue in must_have:
            if eachvalue in ftemp[i]:
                must_have_counter = must_have_counter + 1
                break
        if must_have_counter == 0:
            ftemp.pop(i)
            ftempwithcount.pop(i)
            i = i - 1
        i = i + 1

    print("for the count")
    print(ftempwithcount)
    print(ftemp)

    # Generating Frequency set with tail count
    count = 0
    freqsetwithtail = []
    ftemp0 = ftempwithcount
    for ftemp1 in ftemp0:
        ftemp2 = ftemp1[0][1:]
        ftemp3 = []
        for transaction in transactionList:
            if set(ftemp2) < set(transaction):
                count = count + 1
        ftemp3.append(ftemp1)
        ftemp3.append(count)
        freqsetwithtail.append(ftemp3)
        count = 0
    print("Frequent set with tail count")
    print(freqsetwithtail)
    print("Generated Frequency Set --- F" + str(k))
    if(len(freqsetwithtail) > 0):
        f.append(freqsetwithtail)
    print(f)
    k = k + 1
import os
os.chdir(r'C:\Users\Ashwin PC\Anaconda3\Scripts\Text files')
outputfile = open(r'outputpatterns.txt','w+')

i = 0
while(i<len(f)):
    outputfile.write('Frequent '+str(i + 1)+'-itemsets\n')
    outputfile.write('\n')
    if i == 0:
        j = 0
        while j < len(f[i]):
            outputfile.write('\t'+str(f[i][j][1])+' : {'+str(f[i][j][0])+'}\n')
            j = j + 1
    else:
        j = 0
        while j < len(f[i]):
            outputfile.write('\t'+str(f[i][j][0][1])+' : {'+str(f[i][j][0][0]).replace("[","").replace("]","").replace("'","")+'}\n')
            outputfile.write('Tail count ='+str(f[i][j][1])+'\n')
            j = j + 1
    outputfile.write('\n\tTotal number of frequent '+str(i + 1)+'-itemsets = '+str(len(f[i]))+'\n\n')
    i = i + 1

outputfile.close()


MIS_List
[['1', 0.1], ['2', 0.25], ['3', 0.01], ['4', 0.2], ['5', 0.1], ['6', 0.45], ['7', 0.05], ['8', 0.1], ['9', 0.1], ['10', 0.2], ['11', 0.2], ['12', 0.05], ['13', 0.1], ['14', 0.02], ['15', 0.05], ['16', 0.25], ['17', 0.3], ['18', 0.15], ['19', 0.1], ['20', 0.1], ['21', 0.02], ['22', 0.15], ['23', 0.15], ['24', 0.1], ['25', 0.1], ['26', 0.05], ['27', 0.05], ['28', 0.25], ['29', 0.01], ['30', 0.02], ['31', 0.15], ['32', 0.1], ['33', 0.05], ['34', 0.15], ['35', 0.05], ['36', 0.01], ['37', 0.4], ['38', 0.05], ['39', 0.25], ['40', 0.1], ['41', 0.05], ['42', 0.1], ['43', 0.1], ['44', 0.02], ['45', 0.05], ['46', 0.3], ['47', 0.2], ['48', 0.05], ['49', 0.1], ['50', 0.05]]
support_List
[['1', 0.1], ['2', 0.32], ['3', 0.01], ['4', 0.26], ['5', 0.19], ['6', 0.55], ['7', 0.06], ['8', 0.19], ['9', 0.09], ['10', 0.28], ['11', 0.26], ['12', 0.06], ['13', 0.1], ['14', 0.02], ['15', 0.04], ['16', 0.31], ['17', 0.56], ['18', 0.14], ['19', 0.46], ['20', 0.09], ['21', 0.02], ['22', 0.14], ['23', 0.

for the count
[[['26', '33'], 93], [['26', '5'], 60], [['26', '19'], 127], [['26', '32'], 82], [['26', '40'], 61], [['26', '43'], 155], [['26', '23'], 79], [['26', '4'], 71], [['26', '10'], 84], [['26', '11'], 77], [['26', '47'], 80], [['26', '2'], 83], [['26', '16'], 88], [['26', '28'], 125], [['26', '39'], 59], [['26', '17'], 159], [['26', '46'], 60], [['26', '37'], 126], [['26', '6'], 150], [['33', '50'], 54], [['33', '5'], 68], [['33', '8'], 65], [['33', '19'], 163], [['33', '32'], 116], [['33', '40'], 89], [['33', '43'], 166], [['33', '22'], 51], [['33', '23'], 111], [['33', '4'], 89], [['33', '10'], 103], [['33', '11'], 93], [['33', '47'], 100], [['33', '2'], 104], [['33', '16'], 122], [['33', '28'], 156], [['33', '39'], 83], [['33', '17'], 200], [['33', '46'], 103], [['33', '37'], 165], [['33', '6'], 195], [['50', '19'], 78], [['50', '43'], 75], [['50', '10'], 50], [['50', '2'], 53], [['50', '28'], 65], [['50', '37'], 60], [['50', '6'], 84], [['5', '43'], 100], [['5', '17'], 103

for the count
[[['26', '33', '17'], 58], [['26', '33', '6'], 55], [['26', '19', '43'], 79], [['26', '19', '28'], 60], [['26', '19', '17'], 85], [['26', '19', '37'], 58], [['26', '19', '6'], 73], [['26', '43', '10'], 50], [['26', '43', '47'], 53], [['26', '43', '28'], 77], [['26', '43', '17'], 94], [['26', '43', '37'], 70], [['26', '43', '6'], 78], [['26', '23', '6'], 55], [['26', '10', '6'], 55], [['26', '2', '6'], 54], [['26', '16', '17'], 55], [['26', '16', '6'], 56], [['33', '19', '32'], 62], [['33', '19', '43'], 78], [['33', '19', '10'], 53], [['33', '19', '16'], 57], [['33', '19', '28'], 75], [['33', '19', '17'], 89], [['33', '19', '37'], 73], [['33', '19', '6'], 92], [['33', '32', '43'], 50], [['33', '32', '28'], 53], [['33', '32', '17'], 59], [['33', '32', '37'], 57], [['33', '32', '6'], 54], [['33', '40', '37'], 51], [['33', '43', '10'], 52], [['33', '43', '47'], 51], [['33', '43', '16'], 56], [['33', '43', '28'], 77], [['33', '43', '17'], 101], [['33', '43', '37'], 74], [['33'

Level K > 2 Candidate Generation
[['26', '19', '43', '28'], ['26', '19', '43', '17'], ['26', '19', '43', '37'], ['26', '19', '43', '6'], ['26', '43', '10', '6'], ['33', '19', '32', '43'], ['33', '19', '32', '28'], ['33', '19', '32', '17'], ['33', '19', '32', '37'], ['33', '19', '32', '6'], ['33', '19', '43', '10'], ['33', '19', '43', '16'], ['33', '19', '43', '28'], ['33', '19', '43', '17'], ['33', '19', '43', '37'], ['33', '19', '43', '6'], ['33', '19', '10', '17'], ['33', '19', '10', '6'], ['33', '19', '16', '17'], ['33', '19', '16', '37'], ['33', '19', '16', '6'], ['33', '32', '43', '28'], ['33', '32', '43', '17'], ['33', '32', '43', '37'], ['33', '32', '43', '6'], ['33', '43', '10', '17'], ['33', '43', '10', '6'], ['33', '43', '47', '17'], ['33', '43', '47', '37'], ['33', '43', '47', '6'], ['33', '43', '16', '17'], ['33', '43', '16', '37'], ['33', '43', '16', '6']]
for the count
[[['26', '19', '43', '17'], 52]]
[['26', '19', '43', '17']]
Frequent set with tail count
[[[['26', '19',

In [9]:

import os
os.chdir(r'C:\Users\Ashwin PC\Anaconda3\Scripts\Text files')
outputfile = open(r'outputpatterns.txt','w+')

i = 0
while(i<len(f)):
    outputfile.write('Frequent '+str(i + 1)+'-itemsets\n')
    outputfile.write('\n')
    if i == 0:
        j = 0
        while j < len(f[i]):
            outputfile.write('\t'+str(f[i][j][1])+' : {'+str(f[i][j][0])+'}\n')
            j = j + 1
    else:
        j = 0
        while j < len(f[i]):
            outputfile.write('\t'+str(f[i][j][0][1])+' : {'+str(f[i][j][0][0]).replace("[","").replace("]","").replace("'","")+'}\n')
            outputfile.write('Tail count ='+str(f[i][j][1])+'\n')
            j = j + 1
    outputfile.write('\n\tTotal number of frequent '+str(i + 1)+'-itemsets = '+str(len(f[i]))+'\n\n')
    i = i + 1

outputfile.close()




In [14]:
print(len(f[1]))
print(f[1][0][1])

94
353
